In [13]:
import argparse
import os
import torch
import wandb
import datetime
import pytorch_lightning as L

from glue_transfomer import GLUETransformer
from data_module import GLUEDataModule
from wandb_utils import setup_wandb

#from datetime import datetime



In [14]:
wandb.login()

hyperparams = {
    'learning_rate' : 1e-5,
    'warmup_steps': 0,
    'weight_decay': 0.0,
    'train_batch_size': 32,
    'eval_batch_size': 32,
    'use_cyclic_lr': True,
    'base_lr': 1e-5,
    'max_lr': 1e-1,
    'step_size_up': 100000,
    'step_size_down': 100000
}

model_name = "distilbert-base-uncased"
task_name = 'mrpc'
project_name = 'mlops'

epochs = 3

wandb_logger, checkpoint_callback, log_dir, checkpoint_dir = setup_wandb(hyperparams, model_name, task_name, project_name)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [15]:
L.seed_everything(42)

dm = GLUEDataModule(
    model_name_or_path=model_name,
    task_name=task_name,
)
dm.setup("fit")
model = GLUETransformer(
    model_name_or_path=model_name,
    num_labels=dm.num_labels,
    eval_splits=dm.eval_splits,
    task_name=dm.task_name,
    **hyperparams
)

trainer = L.Trainer(
    max_epochs=epochs,
    accelerator="auto",
    devices=1,
    logger=wandb_logger,
    log_every_n_steps=10,
    default_root_dir=log_dir, 
    callbacks=[checkpoint_callback],
)
trainer.fit(model, datamodule=dm)
wandb.finish()

Seed set to 42


Map:   0%|          | 0/408 [00:00<?, ? examples/s]

/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2829: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


accuracy,▁██
epoch,▁▅█
f1,▁▇█
trainer/global_step,▁▅█
training_loss,█▇▇▇▆█▆█▆▇▆▇▆▆▅▅▅▆▆▅▅▇▅▆▄▄▆▄▃▄▂▄▄▂▁▃▂▃▃▂
val_loss,██▄▅▅▅▆▆▆▆▆▅▆▅▅▃▃▄▃▃▅▄▄▄▃▄▅▄▂▃▁▅▅▄▄▃▄▆▆▄
accuracy,0.83578
epoch,2
f1,0.88388
trainer/global_step,344
training_loss,0.15354


In [ ]:
wandb.login()

hyperparams = {
    'use_cyclic_lr':True,
    'base_lr': 1e-5,
    'max_lr': 1e-1,
    'step_size_up': 100000,
    'step_size_down':  100000
}

task_name = 'mrpc'
model_name = "distilbert-base-uncased"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
project_name = 'mlops'

hyperparam_names = "-".join(hyperparams.keys())
hyperparam_values = "-".join([str(value) for value in hyperparams.values()])
hyperparam_string = f"{hyperparam_names}/{hyperparam_values}"

experiment_name = f'project_1/week_2/{hyperparam_string}-{timestamp}'
folder_structure = f'mlops/project_1/week_2/{hyperparam_string}/'

wandb_experiment_name = experiment_name.replace("/", "-")
wandb_hyperparam_string = hyperparam_string.replace("/", "-")

epochs = 3  # do not change this

log_dir = os.path.join(folder_structure, 'logs')
checkpoint_dir = os.path.join(folder_structure, 'checkpoints')
os.makedirs(log_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

wandb.init(
    project=project_name,
    name=wandb_experiment_name,
    #name = experiment_name,
    config={  
        **hyperparams,  # Pass all hyperparameters dynamically
        "model_name": model_name,
        "task_name": task_name,
    },
    tags=[task_name, model_name],  # Optional tags
    dir=log_dir,  # Use custom log directory for WandB logs
    id=f"{wandb_hyperparam_string}",  # Custom ID using hyperparameter string
)

wandb_logger = WandbLogger(
    project=project_name,
    name=experiment_name,
    log_model=True,  # Log model checkpoints to W&B
    save_dir=log_dir,  # Path for logs
    id=f"{hyperparam_string}"
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  
    dirpath=checkpoint_dir, 
    filename=hyperparam_string + '-{epoch:02d}-{val_loss:.2f}',  
    save_top_k=1,  
    mode='min',  
)

In [ ]:
hyperparam_string

In [ ]:
wandb_experiment_name